In [11]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [30]:
def readData(location):
  if(location == 1):
    y_train_obs_A = pd.read_parquet('data/A/train_targets.parquet')
    X_test_est_A = pd.read_parquet('data/A/X_test_estimated.parquet')
    X_train_est_A = pd.read_parquet('data/A/X_train_estimated.parquet')
    X_train_obs_A = pd.read_parquet('data/A/X_train_observed.parquet')
    return y_train_obs_A, X_test_est_A, X_train_est_A, X_train_obs_A
  elif(location == 2):
    y_train_obs_B = pd.read_parquet('data/B/train_targets.parquet')
    X_test_est_B = pd.read_parquet('data/B/X_test_estimated.parquet' )
    X_train_est_B = pd.read_parquet('data/B/X_train_estimated.parquet')
    X_train_obs_B = pd.read_parquet('data/B/X_train_observed.parquet')
    return y_train_obs_B, X_test_est_B , X_train_est_B, X_train_obs_B
  else:
    y_train_obs_C = pd.read_parquet('data/C/train_targets.parquet')
    X_test_est_C = pd.read_parquet('data/C/X_test_estimated.parquet')
    X_train_est_C = pd.read_parquet('data/C/X_train_estimated.parquet')
    X_train_obs_C = pd.read_parquet('data/C/X_train_observed.parquet')
    X_c = pd.concat([X_train_est_C, X_train_obs_C])
    return y_train_obs_C, X_test_est_C, X_train_est_C, X_train_obs_C

In [31]:
y_a, X_test_A, X_train_A_est, X_train_A_obs = readData(1)

y_b, X_test_B, X_train_B_est, X_train_B_obs = readData(2)
y_c, X_test_C, X_train_C_est, X_train_C_obs = readData(3)

In [32]:
X_train_A_obs.set_index('date_forecast', inplace=True)

X_train_A_est.set_index('date_forecast', inplace=True)

X_test_A.set_index('date_forecast', inplace=True)

y_a.set_index('time', inplace=True)
X_test_A.shape

(2880, 46)

In [33]:
X_train_B_obs.set_index('date_forecast', inplace=True)

X_train_B_est.set_index('date_forecast', inplace=True)

X_test_B.set_index('date_forecast', inplace=True)

y_b.set_index('time', inplace=True)
X_test_B.shape

(2880, 46)

In [34]:
X_train_C_obs.set_index('date_forecast', inplace=True)

X_train_C_est.set_index('date_forecast', inplace=True)

X_test_C.set_index('date_forecast', inplace=True)

y_c.set_index('time', inplace=True)
X_test_C.shape

(2880, 46)

In [35]:
X_test_A.isnull().mean()

date_calc                         0.000000
absolute_humidity_2m:gm3          0.000000
air_density_2m:kgm3               0.000000
ceiling_height_agl:m              0.275347
clear_sky_energy_1h:J             0.000000
clear_sky_rad:W                   0.000000
cloud_base_agl:m                  0.103472
dew_or_rime:idx                   0.000000
dew_point_2m:K                    0.000000
diffuse_rad:W                     0.000000
diffuse_rad_1h:J                  0.000000
direct_rad:W                      0.000000
direct_rad_1h:J                   0.000000
effective_cloud_cover:p           0.000000
elevation:m                       0.000000
fresh_snow_12h:cm                 0.000000
fresh_snow_1h:cm                  0.000000
fresh_snow_24h:cm                 0.000000
fresh_snow_3h:cm                  0.000000
fresh_snow_6h:cm                  0.000000
is_day:idx                        0.000000
is_in_shadow:idx                  0.000000
msl_pressure:hPa                  0.000000
precip_5min

In [36]:
def make_hourly(unsampled_df, target_df):
    sampled_df = unsampled_df.resample('H').mean()
    sampled_df.index = sampled_df.index.floor('H')
    merged_df = sampled_df.merge(target_df, left_index=True, right_index=True, how='inner')
    return merged_df

### A

In [37]:
merged_df_train_obs_A = make_hourly(X_train_A_obs, y_a)
# Reimposta l'indice se lo desideri
merged_df_train_est_A = make_hourly(X_train_A_est, y_a)
print(merged_df_train_obs_A.shape)
print(merged_df_train_est_A.shape)


(29667, 46)
(4418, 46)


C:\Users\claxl\AppData\Local\Temp\ipykernel_26728\3041205111.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  sampled_df = unsampled_df.resample('H').mean()


In [38]:
X_test_A = X_test_A.fillna(0)

X_test_A = X_test_A.resample('H').mean()
X_test_A.index = X_test_A.index.floor('H')

X_test_A = X_test_A.dropna()
X_test_A

C:\Users\claxl\AppData\Local\Temp\ipykernel_26728\1718057266.py:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  X_test_A = X_test_A.resample('H').mean()


,absolute_humidity_2m:gm3,air_density_2m:kgm3,ceiling_height_agl:m,clear_sky_energy_1h:J,clear_sky_rad:W,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,diffuse_rad_1h:J,...,sun_azimuth:d,sun_elevation:d,super_cooled_liquid_water:kgm2,t_1000hPa:K,total_cloud_cover:p,visibility:m,wind_speed_10m:ms,wind_speed_u_10m:ms,wind_speed_v_10m:ms,wind_speed_w_1000hPa:ms
date_forecast,,,,,,,,,,,,,,,,,,,,,
2023-05-01 00:00:00,4.325,1.28675,912.700012,0.000000,0.000000,1061.550049,0.0,271.650024,0.000000,0.000000,...,16.026501,-10.54100,0.000,273.799988,74.949997,29907.500000,3.950,2.100,3.350,0.0
2023-05-01 01:00:00,4.275,1.28600,1111.574951,0.000000,0.000000,1075.100098,0.0,271.450012,0.000000,0.000000,...,30.497250,-7.89450,0.000,273.799988,77.474998,29519.074219,3.825,1.925,3.300,0.0
2023-05-01 02:00:00,4.150,1.28375,1791.300049,0.000000,0.000000,1200.400024,0.0,271.049988,0.000000,0.000000,...,44.517250,-3.81550,0.000,273.849976,88.099998,31009.125000,3.650,1.750,3.200,0.0
2023-05-01 03:00:00,4.025,1.28200,2312.875000,10124.424805,11.675000,1179.849976,0.0,270.649994,9.375000,16845.226562,...,58.083000,1.41250,0.000,273.899994,68.599998,34552.500000,3.500,1.450,3.150,0.0
2023-05-01 04:00:00,3.900,1.28100,2198.299805,141748.593750,76.875000,920.049988,0.0,270.375000,47.400002,102209.703125,...,71.341003,7.46850,0.000,273.924988,66.300003,35483.875000,3.325,1.300,3.050,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-07-03 19:00:00,8.350,1.19725,3638.899902,477093.187500,85.099998,2013.750000,0.0,281.575012,33.625000,168774.546875,...,306.687012,8.15300,0.100,287.024994,86.150002,44056.375000,2.450,2.075,-1.350,0.0
2023-07-03 20:00:00,8.525,1.20050,3552.100098,184335.671875,24.799999,1610.900024,0.0,281.850006,14.325000,86321.125000,...,319.704010,3.26800,0.000,286.600006,75.324997,44017.175781,2.450,2.100,-1.250,0.0
2023-07-03 21:00:00,8.825,1.20450,1736.250000,37429.328125,1.275000,1622.800049,0.0,282.325012,1.300000,28169.150391,...,333.040009,-0.44325,0.000,286.100006,76.849998,43302.050781,2.575,2.150,-1.400,0.0


### B

In [11]:
merged_df_train_obs_B = make_hourly(X_train_B_obs, y_b)
merged_df_train_est_B = make_hourly(X_train_B_est, y_b)
print(merged_df_train_obs_B.shape)
print(merged_df_train_est_B.shape)


(29222, 46)
(3625, 46)


C:\Users\claxl\AppData\Local\Temp\ipykernel_12432\3041205111.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  sampled_df = unsampled_df.resample('H').mean()


In [39]:
X_test_B = X_test_B.fillna(0)

X_test_B = X_test_B.resample('H').mean()
X_test_B.index = X_test_B.index.floor('H')
X_test_B = X_test_B.dropna()

X_test_B.shape

C:\Users\claxl\AppData\Local\Temp\ipykernel_26728\1178095122.py:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  X_test_B = X_test_B.resample('H').mean()


(720, 45)

### C

In [13]:
merged_df_train_obs_C = make_hourly(X_train_C_obs, y_c)
merged_df_train_est_C = make_hourly(X_train_C_est, y_c)
print(merged_df_train_obs_C.shape)
print(merged_df_train_est_C.shape)


(29200, 46)
(2954, 46)


C:\Users\claxl\AppData\Local\Temp\ipykernel_12432\3041205111.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  sampled_df = unsampled_df.resample('H').mean()


In [40]:
X_test_C = X_test_C.fillna(0)

X_test_C = X_test_C.resample('H').mean()
X_test_C.index = X_test_C.index.floor('H')
X_test_C = X_test_C.dropna()

X_test_C.shape

C:\Users\claxl\AppData\Local\Temp\ipykernel_26728\3087904568.py:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  X_test_C = X_test_C.resample('H').mean()


(720, 45)

### SAVE

In [43]:
base = "/Users/claxl/Documents/GitHub/MLProject/Data Exploration/EDA/clearData/"

In [44]:
#merged_df_train_obs_A.to_csv(base + 'A/train_obs_A.csv',index = True)
#merged_df_train_est_A.to_csv(base + 'A/train_est_A.csv',index = True)
X_test_A.to_csv(base + 'X_test_A.csv',index = True)

In [45]:
#merged_df_train_obs_B.to_csv(base + 'B/train_obs_B.csv',index = True)
#merged_df_train_est_B.to_csv(base + 'B/train_est_B.csv',index = True)
X_test_B.to_csv(base + 'B/X_test_B.csv',index = True)

In [46]:
#merged_df_train_obs_C.to_csv(base + 'C/train_obs_C.csv',index = True)
#merged_df_train_est_C.to_csv(base + 'C/train_est_C.csv',index = True)
X_test_C.to_csv(base + 'C/X_test_C.csv',index = True)